In [1]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os

pd.set_option('display.max_rows', 200)

In [2]:
%load_ext dotenv
%dotenv ../.env

In [3]:
con = create_engine(os.getenv('BUILD_ENGINE').replace('localhost', os.getenv('IP')))

In [4]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [5]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [6]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [7]:
# Specify latest PLUTO version
pluto = 'pluto'
# Specify previous PLUTO version or the one you want to compare to
plutoprev = 'dcp_pluto'

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [8]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [9]:
df = pd.read_sql(sql=sql, con=con)

In [10]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,2561644.0,NaN,NaN
1,None,None,NaN,2560881.0,NaN
2,C1-9A,C1-9A,137.0,134.0,3.0
115,C1-9,C1-9,999.0,1000.0,-1.0
116,R5B,R5B,31642.0,31643.0,-1.0
117,R10A,R10A,922.0,923.0,-1.0
118,C6-2,C6-2,797.0,798.0,-1.0
119,C4-2,C4-2,1677.0,1678.0,-1.0
120,C5-3,C5-3,655.0,656.0,-1.0
121,C4-4D,C4-4D,347.0,348.0,-1.0


### Commercial overlay

In [11]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [12]:
df = pd.read_sql(sql=sql, con=con)

In [13]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1587.0,1587.0,0.0
1,C1-2,C1-2,11031.0,11031.0,0.0
2,C1-3,C1-3,13786.0,13790.0,-4.0
3,C1-4,C1-4,8095.0,8097.0,-2.0
4,C1-5,C1-5,2540.0,2547.0,-7.0
5,C2-1,C2-1,1684.0,1684.0,0.0
6,C2-2,C2-2,6023.0,6023.0,0.0
7,C2-3,C2-3,11942.0,11952.0,-10.0
8,C2-4,C2-4,16714.0,16727.0,-13.0
9,C2-5,C2-5,1523.0,1523.0,0.0


### Special purpose district

In [14]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [15]:
df = pd.read_sql(sql=sql, con=con)

In [16]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,2477048.0,NaN,NaN
1,None,None,NaN,2476583.0,NaN
64,EC-6,EC-6,409.0,410.0,-1.0
65,HY,HY,288.0,289.0,-1.0
66,EC-5,EC-5,558.0,559.0,-1.0
67,MX-6,MX-6,11.0,12.0,-1.0
68,OP,OP,13442.0,13443.0,-1.0
69,LI,LI,550.0,551.0,-1.0
70,CL,CL,1333.0,1334.0,-1.0
71,MX-13,MX-13,85.0,86.0,-1.0


In [17]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [18]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [19]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-20', 'MX-16/EC-6', 'MX-16/EC-5', 'BSC', 'SW']

### Limited height

In [20]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

In [ ]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

### Building class

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df[df['diff'] != 0]

In [ ]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

In [ ]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

### Land use

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df[df['diff'] != 0]

In [ ]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

In [ ]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

### Owner type

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

In [ ]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

### Area source

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

In [ ]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

### Extension

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

In [ ]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

### Proxy code

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

In [ ]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

### Lot type

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

In [ ]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

### Basement code

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

In [ ]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

### PLUTO map id

In [ ]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [ ]:
df = pd.read_sql(sql=sql, con=con)

In [ ]:
df

In [ ]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [ ]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

In [ ]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

#### done